# Feature extraction using LSTM (Original future -> Improve future)

---



Ở giai đoạn Feature extraction using LSTM, chúng ta sẽ sử dụng mô hình LSTM để trích xuất các đặc trưng từ dữ liệu đầu vào. Đầu ra của giai đoạn này sẽ là các đặc trưng đã được trích xuất, được biểu diễn dưới dạng một ma trận kích thước (số lượng mẫu, số chiều đặc trưng), trong đó mỗi hàng đại diện cho một mẫu dữ liệu và mỗi cột đại diện cho một đặc trưng.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
# Đọc tập dữ liệu
df = pd.read_csv('/content/drive/MyDrive/NT204_IDPS/UNSW-NB15_1_label.csv')

<ipython-input-3-807a0538a79e>:2: DtypeWarning: Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/NT204_IDPS/UNSW-NB15_1_label.csv')


In [ ]:
df

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
0,59.166.0.0,1390,149.171.126.6,53,udp,CON,0.001055,132,164,31,...,0,3,7,1,3,1,1,1,NaN,0
1,59.166.0.0,33661,149.171.126.9,1024,udp,CON,0.036133,528,304,31,...,0,2,4,2,3,1,1,2,NaN,0
2,59.166.0.6,1464,149.171.126.7,53,udp,CON,0.001119,146,178,31,...,0,12,8,1,2,2,1,1,NaN,0
3,59.166.0.5,3593,149.171.126.5,53,udp,CON,0.001209,132,164,31,...,0,6,9,1,1,1,1,1,NaN,0
4,59.166.0.3,49664,149.171.126.0,53,udp,CON,0.001169,146,178,31,...,0,7,9,1,1,1,1,1,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699996,59.166.0.8,12520,149.171.126.6,31010,tcp,FIN,0.020383,320,1874,31,...,0,8,20,7,5,1,1,4,NaN,0
699997,59.166.0.0,18895,149.171.126.9,80,tcp,FIN,1.402957,19410,1087890,31,...,0,1,1,2,7,2,2,2,NaN,0
699998,59.166.0.0,30103,149.171.126.5,5190,tcp,FIN,0.007108,2158,2464,31,...,0,13,13,6,7,2,1,2,NaN,0
699999,59.166.0.6,30388,149.171.126.5,111,udp,CON,0.004435,568,304,31,...,0,10,13,6,5,1,1,3,NaN,0


In [ ]:
df.dtypes

In [ ]:
# Chuyển đổi các giá trị categorical thành dạng số
cat_cols = df.select_dtypes(include='object').columns
for col in cat_cols:
    df[col] = pd.Categorical(df[col])
    df[col] = df[col].cat.codes

In [ ]:
df

In [ ]:
# Thay thế giá trị NaN trong các cột numerical bằng mean của cột đó
num_cols = df.select_dtypes(include=np.number).columns
for col in num_cols:
    df[col] = df[col].fillna(df[col].mean())

In [ ]:
df

In [ ]:
# Chuẩn hóa MixMaxScaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[df.columns] = scaler.fit_transform(df[df.columns])

In [ ]:
df

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
0,0.769231,0.003949,0.511628,0.918046,0.895522,0.133333,1.200687e-07,0.000010,0.000011,0.121569,...,0.0,0.046512,0.146341,0.000000,0.040816,0.000000,0.000000,0.000000,0.0,0.0
1,0.769231,0.325714,0.581395,0.690519,0.895522,0.133333,4.112267e-06,0.000039,0.000021,0.121569,...,0.0,0.023256,0.073171,0.024390,0.040816,0.000000,0.000000,0.027027,0.0,0.0
2,0.923077,0.004687,0.534884,0.918046,0.895522,0.133333,1.273525e-07,0.000011,0.000012,0.121569,...,0.0,0.255814,0.170732,0.000000,0.020408,0.028571,0.000000,0.000000,0.0,0.0
3,0.897436,0.025920,0.488372,0.918046,0.895522,0.133333,1.375953e-07,0.000010,0.000011,0.121569,...,0.0,0.116279,0.195122,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
4,0.846154,0.485285,0.139535,0.918046,0.895522,0.133333,1.330429e-07,0.000011,0.000012,0.121569,...,0.0,0.139535,0.195122,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699996,0.974359,0.114941,0.511628,0.321107,0.850746,0.333333,2.319772e-06,0.000023,0.000128,0.121569,...,0.0,0.162791,0.463415,0.146341,0.081633,0.000000,0.000000,0.081081,0.0,0.0
699997,0.769231,0.178520,0.581395,0.000102,0.850746,0.333333,1.596694e-04,0.001419,0.074231,0.121569,...,0.0,0.000000,0.000000,0.024390,0.122449,0.028571,0.030303,0.027027,0.0,0.0
699998,0.769231,0.290229,0.488372,0.045651,0.850746,0.333333,8.089556e-07,0.000158,0.000168,0.121569,...,0.0,0.279070,0.292683,0.121951,0.122449,0.028571,0.000000,0.027027,0.0,0.0
699999,0.923077,0.293072,0.488372,0.000148,0.895522,0.133333,5.047437e-07,0.000042,0.000021,0.121569,...,0.0,0.209302,0.292683,0.121951,0.081633,0.000000,0.000000,0.054054,0.0,0.0


In [ ]:
# Tách các cột dữ liệu và nhãn ra
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [ ]:
X

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat
0,0.769231,0.003949,0.511628,0.918046,0.895522,0.133333,1.200687e-07,0.000010,0.000011,0.121569,...,0.0,0.0,0.046512,0.146341,0.000000,0.040816,0.000000,0.000000,0.000000,0.0
1,0.769231,0.325714,0.581395,0.690519,0.895522,0.133333,4.112267e-06,0.000039,0.000021,0.121569,...,0.0,0.0,0.023256,0.073171,0.024390,0.040816,0.000000,0.000000,0.027027,0.0
2,0.923077,0.004687,0.534884,0.918046,0.895522,0.133333,1.273525e-07,0.000011,0.000012,0.121569,...,0.0,0.0,0.255814,0.170732,0.000000,0.020408,0.028571,0.000000,0.000000,0.0
3,0.897436,0.025920,0.488372,0.918046,0.895522,0.133333,1.375953e-07,0.000010,0.000011,0.121569,...,0.0,0.0,0.116279,0.195122,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
4,0.846154,0.485285,0.139535,0.918046,0.895522,0.133333,1.330429e-07,0.000011,0.000012,0.121569,...,0.0,0.0,0.139535,0.195122,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699996,0.974359,0.114941,0.511628,0.321107,0.850746,0.333333,2.319772e-06,0.000023,0.000128,0.121569,...,0.0,0.0,0.162791,0.463415,0.146341,0.081633,0.000000,0.000000,0.081081,0.0
699997,0.769231,0.178520,0.581395,0.000102,0.850746,0.333333,1.596694e-04,0.001419,0.074231,0.121569,...,0.0,0.0,0.000000,0.000000,0.024390,0.122449,0.028571,0.030303,0.027027,0.0
699998,0.769231,0.290229,0.488372,0.045651,0.850746,0.333333,8.089556e-07,0.000158,0.000168,0.121569,...,0.0,0.0,0.279070,0.292683,0.121951,0.122449,0.028571,0.000000,0.027027,0.0
699999,0.923077,0.293072,0.488372,0.000148,0.895522,0.133333,5.047437e-07,0.000042,0.000021,0.121569,...,0.0,0.0,0.209302,0.292683,0.121951,0.081633,0.000000,0.000000,0.054054,0.0


In [ ]:
y

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
699996    0.0
699997    0.0
699998    0.0
699999    0.0
700000    0.0
Name: Label, Length: 700001, dtype: float64

In [ ]:
# Chuyển đổi nhãn thành dạng one-hot encoding
y = tf.keras.utils.to_categorical(y)

In [ ]:
y

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [ ]:
y.shape

In [ ]:
X.shape

In [ ]:
X = df.values.reshape(df.shape[0], 1, df.shape[1])

In [ ]:
X

array([[[0.76923077, 0.00394938, 0.51162791, ..., 0.        ,
         0.        , 0.        ]],

       [[0.76923077, 0.32571383, 0.58139535, ..., 0.02702703,
         0.        , 0.        ]],

       [[0.92307692, 0.00468739, 0.53488372, ..., 0.        ,
         0.        , 0.        ]],

       ...,

       [[0.76923077, 0.29022928, 0.48837209, ..., 0.02702703,
         0.        , 0.        ]],

       [[0.92307692, 0.29307164, 0.48837209, ..., 0.05405405,
         0.        , 0.        ]],

       [[0.76923077, 0.05047422, 0.48837209, ..., 0.02702703,
         0.        , 0.        ]]])

In [ ]:
X.shape

(700001, 1, 49)

In [ ]:
# Xây dựng mô hình LSTM
model_lstm = tf.keras.Sequential([
    tf.keras.layers.LSTM(64, input_shape=(1, X.shape[2]), return_sequences=True),
    tf.keras.layers.LSTM(32) # Trích xuất ra 32 thuộc tính từ 50 thuộc tính ban đầu 
])
model_lstm.compile(optimizer='adam',  loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model_lstm.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 64)             29184     
                                                                 
 lstm_1 (LSTM)               (None, 32)                12416     
                                                                 
Total params: 41,600
Trainable params: 41,600
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Output of LSTM model. output là đầu ra của mô hình với kích thước (số lượng mẫu, số chiều đặc trưng), trong trường hợp này số chiều đặc trưng là 64 (số units của lớp LSTM cuối cùng trong mô hình).
lstm_output = model_lstm.predict(X)

21876/21876 [==============================] - 56s 3ms/step


In [ ]:
print(lstm_output)

[[ 0.00078437 -0.00104939  0.00151247 ... -0.00279095  0.01793722
  -0.00458832]
 [-0.00172908 -0.00023229 -0.00148056 ... -0.00360715  0.01487223
  -0.00271759]
 [-0.00030332 -0.00147978 -0.00085849 ... -0.00454064  0.0180383
  -0.00507256]
 ...
 [-0.0001355   0.00745667 -0.0110331  ... -0.01515107  0.018881
   0.01503836]
 [-0.00267006  0.00360023 -0.00752025 ... -0.00954908  0.01849387
   0.01110809]
 [-0.00151812  0.00641394 -0.00719875 ... -0.01302245  0.02412112
   0.00724998]]


Trong trường hợp này, sau khi áp dụng mô hình LSTM cho dữ liệu đầu vào, đầu ra của mô hình sẽ có kích thước (số lượng mẫu, số chiều đặc trưng), trong đó số chiều đặc trưng là số lượng units trong lớp LSTM cuối cùng trong mô hình.

In [ ]:
# Save features
features_lstm = pd.DataFrame(lstm_output)

In [ ]:
# Tạo một Series mới chứa dữ liệu cho cột mới
label_col = df.iloc[:, -1]

# Thêm cột mới vào DataFrame bằng cách gán giá trị cho loc với tên cột mới
features_lstm.loc[:, 'Label'] = label_col

In [ ]:
features_lstm

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,Label
0,0.000784,-0.001049,0.001512,0.006754,0.011499,-0.004299,0.014558,0.008297,0.006732,-0.010234,...,-0.002586,0.001357,0.002969,0.002249,0.000480,-0.005918,-0.002791,0.017937,-0.004588,0.0
1,-0.001729,-0.000232,-0.001481,0.010096,0.009991,-0.006609,0.009421,0.008801,0.003469,-0.009637,...,-0.002133,0.001367,0.002027,-0.001455,-0.002934,-0.004449,-0.003607,0.014872,-0.002718,0.0
2,-0.000303,-0.001480,-0.000858,0.007755,0.012876,-0.003377,0.014169,0.009359,0.006859,-0.009590,...,-0.002670,0.002738,0.002659,0.002459,-0.000197,-0.005347,-0.004541,0.018038,-0.005073,0.0
3,0.000185,-0.001889,0.000178,0.007013,0.012079,-0.003337,0.014465,0.009616,0.006650,-0.011173,...,-0.002669,0.001856,0.002888,0.002480,0.001038,-0.005272,-0.004271,0.019026,-0.004801,0.0
4,-0.002524,-0.004588,-0.001969,0.004825,0.010097,-0.004221,0.008892,0.012641,0.005904,-0.014286,...,-0.003185,0.001697,0.000659,0.001246,-0.000196,-0.002837,-0.008052,0.019296,-0.002276,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699996,0.004005,0.004057,-0.011647,0.006739,0.006396,0.014973,0.004634,0.021593,0.002073,0.001965,...,-0.019730,0.014775,0.006219,0.000308,0.026500,-0.006310,-0.014300,0.024731,0.014034,0.0
699997,-0.001459,0.012455,-0.000831,0.008925,0.007903,0.005116,0.007587,0.003013,0.006287,0.010071,...,-0.011587,0.018998,0.002898,-0.003317,0.020077,-0.011223,-0.009170,0.018937,0.011260,0.0
699998,-0.000135,0.007457,-0.011033,0.012862,0.007019,0.010241,-0.001597,0.015245,0.001131,0.007177,...,-0.017714,0.014640,0.005649,-0.002440,0.020371,-0.005837,-0.015151,0.018881,0.015038,0.0
699999,-0.002670,0.003600,-0.007520,0.001519,0.004649,-0.000041,0.001512,0.009848,0.002292,0.003942,...,-0.007983,0.003298,0.004833,0.003105,0.010030,0.004066,-0.009549,0.018494,0.011108,0.0


In [ ]:
features_lstm.to_csv('/content/drive/MyDrive/NT204_IDPS/lstm_features.csv', index=False)

# Feature encoding (Bỏ qua bước này do đã tiền xử lý dữ liệu về dạng số ở bước trước)

In [ ]:
features_lstm.dtypes

In [ ]:
from sklearn.preprocessing import OneHotEncoder
# Create a One-Hot Encoder object
encoder = OneHotEncoder()

In [ ]:
features_lstm.columns = features_lstm.columns.astype(str)

In [ ]:
# Fit the encoder to the features and transform the features
encoded_features = encoder.fit_transform(features_lstm)

In [ ]:
# Print the encoded features as a DataFrame
encoded_features = pd.DataFrame(encoded_features, columns=encoder.get_feature_names(features_lstm.columns))

In [ ]:
print(encoded_features)

# Feature matrix

In [175]:
# Extract the features from the DataFrame
features = features_lstm.drop('Label', axis=1).to_numpy()

# Extract the labels from the DataFrame
labels = features_lstm['Label'].to_numpy()

In [176]:
features

array([[ 0.00078437, -0.00104939,  0.00151247, ..., -0.00279095,
         0.01793722, -0.00458832],
       [-0.00172908, -0.00023229, -0.00148056, ..., -0.00360715,
         0.01487223, -0.00271759],
       [-0.00030332, -0.00147978, -0.00085849, ..., -0.00454064,
         0.0180383 , -0.00507256],
       ...,
       [-0.0001355 ,  0.00745667, -0.0110331 , ..., -0.01515107,
         0.018881  ,  0.01503836],
       [-0.00267006,  0.00360023, -0.00752025, ..., -0.00954908,
         0.01849387,  0.01110809],
       [-0.00151812,  0.00641394, -0.00719875, ..., -0.01302245,
         0.02412112,  0.00724998]], dtype=float32)

In [177]:
labels

array([0., 0., 0., ..., 0., 0., 0.])

In [178]:
# Normalize the features
normalized_features = (features - np.mean(features, axis=0)) / np.std(features, axis=0)

In [179]:
normalized_features

array([[ 0.18791759, -1.2823544 ,  2.4153156 , ...,  2.2254918 ,
         0.50167555, -2.3475947 ],
       [-0.55944324, -1.0559587 ,  1.6617535 , ...,  1.9660269 ,
        -0.17267612, -1.9611286 ],
       [-0.13550097, -1.401605  ,  1.8183722 , ...,  1.6692756 ,
         0.5239145 , -2.4476335 ],
       ...,
       [-0.08560001,  1.0744466 , -0.74331856, ..., -1.7037137 ,
         0.70932233,  1.707006  ],
       [-0.8392376 ,  0.0059296 ,  0.14112127, ...,  0.07712553,
         0.62414736,  0.89506817],
       [-0.49671578,  0.7855318 ,  0.22206652, ..., -1.0270375 ,
         1.8622409 ,  0.09803499]], dtype=float32)

In [180]:
# Create the feature matrix with labels
feature_matrix = np.hstack((normalized_features, labels.reshape(-1, 1)))

In [181]:
feature_matrix

array([[ 0.18791759, -1.28235435,  2.41531563, ...,  0.50167555,
        -2.34759474,  0.        ],
       [-0.55944324, -1.05595875,  1.66175354, ..., -0.17267612,
        -1.96112859,  0.        ],
       [-0.13550097, -1.40160501,  1.81837225, ...,  0.52391452,
        -2.4476335 ,  0.        ],
       ...,
       [-0.08560001,  1.07444656, -0.74331856, ...,  0.70932233,
         1.70700598,  0.        ],
       [-0.83923757,  0.0059296 ,  0.14112127, ...,  0.62414736,
         0.89506817,  0.        ],
       [-0.49671578,  0.78553182,  0.22206652, ...,  1.86224091,
         0.09803499,  0.        ]])

In [ ]:
# Shuffle the feature matrix
np.random.shuffle(feature_matrix) # Ko chay dong nay

In [ ]:
feature_matrix

# Train by CNN

In [182]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split

In [183]:
X_cnn = feature_matrix[:, :-1]
y_cnn = feature_matrix[:, -1]

In [184]:
X_cnn

array([[ 0.18791759, -1.28235435,  2.41531563, ...,  2.22549176,
         0.50167555, -2.34759474],
       [-0.55944324, -1.05595875,  1.66175354, ...,  1.9660269 ,
        -0.17267612, -1.96112859],
       [-0.13550097, -1.40160501,  1.81837225, ...,  1.66927564,
         0.52391452, -2.4476335 ],
       ...,
       [-0.08560001,  1.07444656, -0.74331856, ..., -1.70371366,
         0.70932233,  1.70700598],
       [-0.83923757,  0.0059296 ,  0.14112127, ...,  0.07712553,
         0.62414736,  0.89506817],
       [-0.49671578,  0.78553182,  0.22206652, ..., -1.0270375 ,
         1.86224091,  0.09803499]])

In [185]:
y_cnn

array([0., 0., 0., ..., 0., 0., 0.])

In [186]:
# Convert array thành DataFrame
y_cnn = pd.DataFrame(y_cnn, columns=['Class'])

In [187]:
# sử dụng pd.get_dummies() để reshape cột "Class"
y_cnn = pd.get_dummies(y_cnn, columns=['Class'])

In [188]:
# Reshape dữ liệu để phù hợp với mô hình CNN
X_cnn = X_cnn.reshape(X_cnn.shape[0], X_cnn.shape[1], 1)

In [189]:
print(X_cnn.shape)

(700001, 32, 1)


In [190]:
print(y_cnn.shape)

(700001, 2)


In [191]:
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra với tỷ lệ 7:3
X_train, X_test, y_train, y_test = train_test_split(X_cnn, y_cnn, test_size=0.3, random_state=0)

In [ ]:
X_train.shape

(490000, 32, 1)

In [ ]:
X_test.shape

(210001, 32, 1)

In [192]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Conv1D, Dense, Dropout, LSTM, MaxPooling1D, Flatten
from keras.models import Sequential
# Xây dựng mô hình CNN
model_cnn = Sequential()
model_cnn.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model_cnn.add(MaxPooling1D(pool_size=2))
model_cnn.add(Dropout(0.2))
model_cnn.add(Flatten())
model_cnn.add(Dense(units=2, activation='softmax')) # obtains the output probability of each neuron and its fully connected layer weight

In [193]:
# Compile the model
model_cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [194]:
model_cnn.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_6 (Conv1D)           (None, 30, 64)            256       
                                                                 
 max_pooling1d_6 (MaxPooling  (None, 15, 64)           0         
 1D)                                                             
                                                                 
 dropout_6 (Dropout)         (None, 15, 64)            0         
                                                                 
 flatten_5 (Flatten)         (None, 960)               0         
                                                                 
 dense_6 (Dense)             (None, 2)                 1922      
                                                                 
Total params: 2,178
Trainable params: 2,178
Non-trainable params: 0
____________________________________________________

In [195]:
# Huấn luyện mô hình
model_cnn.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Epoch 1/5
15313/15313 [==============================] - 94s 6ms/step - loss: 0.0024 - accuracy: 0.9995 - val_loss: 3.0626e-07 - val_accuracy: 1.0000
Epoch 2/5
15313/15313 [==============================] - 93s 6ms/step - loss: 3.3670e-06 - accuracy: 1.0000 - val_loss: 6.7497e-09 - val_accuracy: 1.0000
Epoch 3/5
15313/15313 [==============================] - 84s 5ms/step - loss: 1.4861e-06 - accuracy: 1.0000 - val_loss: 1.0835e-09 - val_accuracy: 1.0000
Epoch 4/5
15313/15313 [==============================] - 94s 6ms/step - loss: 7.6436e-07 - accuracy: 1.0000 - val_loss: 7.2884e-11 - val_accuracy: 1.0000
Epoch 5/5
15313/15313 [==============================] - 83s 5ms/step - loss: 2.1080e-07 - accuracy: 1.0000 - val_loss: 8.8016e-10 - val_accuracy: 1.0000


# Weight set => Probability

Xem dòng **model_cnn.add(Dense(units=2, activation='softmax'))**

The activation function obtains the output probability of each neuron and its fully connected layer weight. 
Generally, the value of activation function lies between 0 to 1 and a maximum value represents the output label. Figure 
5 depicts the overall process flow of proposed intrusion detection system for an IoT environment.

# Intrusion detection by CNN (Results)

In [196]:
# Đánh giá mô hình trên tập kiểm tra
test_loss, test_acc = model_cnn.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)

6563/6563 [==============================] - 13s 2ms/step - loss: 8.8016e-10 - accuracy: 1.0000
Test accuracy: 1.0


In [197]:
from sklearn.metrics import classification_report
# predict on test set
y_pred_test = model_cnn.predict(X_test)

# calculate precision, recall, and f1-score
print(classification_report(y_test.values.argmax(axis=1), y_pred_test.argmax(axis=1)))

6563/6563 [==============================] - 11s 2ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    203300
           1       1.00      1.00      1.00      6701

    accuracy                           1.00    210001
   macro avg       1.00      1.00      1.00    210001
weighted avg       1.00      1.00      1.00    210001



In [198]:
# predict on X_cnn set
y_pred_total = model_cnn.predict(X_cnn)

21876/21876 [==============================] - 43s 2ms/step


In [199]:
y_pred_total[10000][0]

0.99999994

In [200]:
number_of_normal = 0
number_of_attack = 0
def final_decision_IDS(index):
  if (y_pred_total[index][0] > y_pred_total[index][1]):
    return "Normal"
  else:
    return "Attack"

In [201]:
number_of_packets = len(df)

In [202]:
for i in range(number_of_packets):
  print("Final classification for packet {} is {}".format(i, final_decision_IDS(i)))
  if final_decision_IDS(i) == "Normal":
    number_of_normal += 1
  else:
    number_of_attack += 1

Streaming output truncated to the last 5000 lines.
Final classification for packet 695001 is Normal
Final classification for packet 695002 is Normal
Final classification for packet 695003 is Normal
Final classification for packet 695004 is Normal
Final classification for packet 695005 is Normal
Final classification for packet 695006 is Normal
Final classification for packet 695007 is Normal
Final classification for packet 695008 is Normal
Final classification for packet 695009 is Normal
Final classification for packet 695010 is Normal
Final classification for packet 695011 is Normal
Final classification for packet 695012 is Normal
Final classification for packet 695013 is Normal
Final classification for packet 695014 is Normal
Final classification for packet 695015 is Normal
Final classification for packet 695016 is Normal
Final classification for packet 695017 is Normal
Final classification for packet 695018 is Normal
Final classification for packet 695019 is Normal
Final classificati

In [203]:
print("Number of Normal packets: ", number_of_normal)

Number of Normal packets:  677786


In [204]:
print("Number of Attack packets: ", number_of_attack)

Number of Attack packets:  22215
